In [1]:
import duckdb

# Load the CSV file into DuckDB
con = duckdb.connect()
results = con.sql("SELECT * FROM '../results/naive_bayes_예측결과.csv'")
# Display the first 5 rows of the results
display(results.df().head())

,주_용도_코드,주_용도_코드_명,기타_용도,Count,predicted_label,is_correct,predicted_label_name
0,14000,업무시설,"일반업무시설, 교육연구시설, 제2종근린생활시설",1,04000,0,제2종근린생활시설
1,01000,단독주택,"창고,부속사,주택",1,01000,1,단독주택
2,03000,제1종근린생활시설,"화장실,세면장",1,01000,0,단독주택
3,17000,공장,공작물YT-5,1,01000,0,단독주택
4,04000,제2종근린생활시설,"공장(사무실,창고,숙직실)",1,17000,0,공장


In [2]:
# Calculate the overall accuracy
overall_accuracy = con.sql("""
    SELECT 
        SUM(Count * is_correct) / SUM(Count) AS overall_accuracy
    FROM results
""")
print("Overall Accuracy: {:.2f}%".format(overall_accuracy.df().iloc[0, 0] * 100))

# calculate the accuracy by 주_용도_코드
accuracy = con.sql("""
    SELECT 
        주_용도_코드, 
        주_용도_코드_명, 
        SUM(Count) AS total_count, 
        SUM(Count * is_correct) AS correct_count, 
        SUM(Count * is_correct) / SUM(Count) AS accuracy
    FROM results
    WHERE LENGTH("주_용도_코드") = 5 AND "주_용도_코드" ~ '^\\d+$'
    GROUP BY 주_용도_코드, 주_용도_코드_명
    HAVING SUM(Count) >= 100
    ORDER BY 주_용도_코드
""")
accuracy_df = accuracy.df().astype({"total_count": "int", "correct_count": "int"})
accuracy_df["accuracy"] = accuracy_df["accuracy"].apply(
    lambda x: "{:.2f}%".format(x * 100)
)
display(accuracy_df)
# Save the accuracy results to a CSV file
accuracy_df.to_csv(
    "../results/naive_bayes_accuracy.csv", index=False, encoding="utf-8-sig"
)
accuracy.df().astype({"total_count": "int", "correct_count": "int"}).to_csv(
    "../results/naive_bayes_accuracy_orig.csv", index=False, encoding="utf-8-sig"
)

Overall Accuracy: 84.78%


,주_용도_코드,주_용도_코드_명,total_count,correct_count,accuracy
0,01000,단독주택,4326700,4189356,96.83%
1,02000,공동주택,578273,541520,93.64%
2,03000,제1종근린생활시설,594056,98580,16.59%
3,04000,제2종근린생활시설,718151,499771,69.59%
4,05000,문화및집회시설,15969,11384,71.29%
5,06000,종교시설,40251,28507,70.82%
6,07000,판매시설,12500,7224,57.79%
7,08000,운수시설,3966,2674,67.42%
8,09000,의료시설,7592,6278,82.69%
9,10000,교육연구시설,88934,76632,86.17%


In [3]:
# top and bottom 5 by accuracy
top_5_accuracy = accuracy.df().nlargest(5, "accuracy")
bottom_5_accuracy = accuracy.df().query("accuracy > 0").nsmallest(5, "accuracy")
print("Top 5 Accuracy:")
display(top_5_accuracy)
print("Bottom 5 Accuracy:")
display(bottom_5_accuracy)

Top 5 Accuracy:


,주_용도_코드,주_용도_코드_명,total_count,correct_count,accuracy
0,01000,단독주택,4326700.0,4189356.0,0.968257
1,02000,공동주택,578273.0,541520.0,0.936444
20,21000,동물및식물관련시설,439989.0,408871.0,0.929276
16,17000,공장,404164.0,371022.0,0.917999
29,31000,야영장시설,7232.0,6320.0,0.873894


Bottom 5 Accuracy:


,주_용도_코드,주_용도_코드_명,total_count,correct_count,accuracy
2,03000,제1종근린생활시설,594056.0,98580.0,0.165944
15,16000,위락시설,5482.0,2346.0,0.427946
23,24000,방송통신시설,3343.0,1568.0,0.469040
27,29000,장례시설,518.0,271.0,0.523166
6,07000,판매시설,12500.0,7224.0,0.577920


In [4]:
# Calculate the average Count for is_correct values 0 and 1 grouped by 주_용도_코드 and 주_용도_코드_명
average_count = con.sql("""
	SELECT 
		주_용도_코드, 
		주_용도_코드_명,
		AVG(CASE WHEN is_correct = 1 THEN Count ELSE NULL END) AS avg_count_correct,
		AVG(CASE WHEN is_correct = 0 THEN Count ELSE NULL END) AS avg_count_incorrect,
	FROM results
    WHERE LENGTH("주_용도_코드") = 5 AND "주_용도_코드" ~ '^\\d+$'
	GROUP BY 주_용도_코드, 주_용도_코드_명
    HAVING SUM(Count) >= 100
    ORDER BY 주_용도_코드
""")
average_count_df = average_count.df()
# format the average counts to 2 decimal places
average_count_df["avg_count_correct"] = average_count_df["avg_count_correct"].apply(
    lambda x: "{:.2f}".format(x)
)
average_count_df["avg_count_incorrect"] = average_count_df["avg_count_incorrect"].apply(
    lambda x: "{:.2f}".format(x)
)
display(average_count_df)
# Save the average counts to a CSV file
average_count_df.to_csv(
    "../results/naive_bayes_average_count.csv", index=False, encoding="utf-8-sig"
)

,주_용도_코드,주_용도_코드_명,avg_count_correct,avg_count_incorrect
0,01000,단독주택,115.62,10.47
1,02000,공동주택,22.75,4.06
2,03000,제1종근린생활시설,8.74,20.79
3,04000,제2종근린생활시설,30.10,9.40
4,05000,문화및집회시설,18.13,2.28
5,06000,종교시설,17.60,4.90
6,07000,판매시설,12.08,3.11
7,08000,운수시설,8.83,2.05
8,09000,의료시설,9.14,1.97
9,10000,교육연구시설,23.63,2.92
